In [1]:
import torch
print(torch.cuda.is_available())  # Should print True if CUDA is available
print(torch.cuda.get_device_name(0))  # Prints the name of your GPU


True
NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models.segmentation as models
from torch.utils.data import Dataset, DataLoader 
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")
    print(f"Memory Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"Memory Reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


CUDA Available: True
Device Name: NVIDIA GeForce RTX 3050 Laptop GPU
Memory Allocated: 0.00 GB
Memory Reserved: 0.00 GB
Total Memory: 4.00 GB


In [4]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform_img=None, transform_mask=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform_img = transform_img
        self.transform_mask = transform_mask
        self.images = os.listdir(image_dir)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        
        image = Image.open(img_path).convert("RGB")  # Ensure 3 channels
        mask = Image.open(mask_path).convert("L")  # Convert mask to grayscale
        
        if self.transform_img:
            image = self.transform_img(image)
        if self.transform_mask:
            mask = self.transform_mask(mask)
        
        return image, mask

# Define Separate Transformations
transform_img = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

transform_mask = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: (x > 0).float())  # Binarize the mask (if needed)
])

# Dataset and DataLoader
train_dataset = SegmentationDataset("Kvasir-SEG/images", "Kvasir-SEG/masks", 
                                    transform_img=transform_img, 
                                    transform_mask=transform_mask)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResUnetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResUnetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)

        # Residual connection (if input and output channels are different)
        if in_channels != out_channels:
            self.res_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        residual = self.res_conv(x) if hasattr(self, 'res_conv') else x  # Ensure it's defined
        x = self.conv1(x)
        x = F.relu(self.bn(x))
        x = self.conv2(x)
        x = self.bn(x)
        return F.relu(x + residual)

class ResUnetPlusPlus(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(ResUnetPlusPlus, self).__init__()
        self.encoder1 = ResUnetBlock(in_channels, 64)
        self.encoder2 = ResUnetBlock(64, 128)
        self.encoder3 = ResUnetBlock(128, 256)
        
        self.middle = ResUnetBlock(256, 512)
        
        self.decoder3 = ResUnetBlock(512, 256)
        self.decoder2 = ResUnetBlock(256, 128)
        self.decoder1 = ResUnetBlock(128, 64)
        
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        
        m = self.middle(e3)
        
        d3 = self.decoder3(m) + e3
        d2 = self.decoder2(d3) + e2
        d1 = self.decoder1(d2) + e1
        
        return self.final_conv(d1)

# Now instantiate the model
model = ResUnetPlusPlus(in_channels=3, out_channels=1)


In [6]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# IoU Score
def iou_score(pred, target, smooth=1e-6):
    pred = torch.sigmoid(pred) > 0.5
    target = target > 0.5  # Ensure target is also a binary mask
    intersection = (pred.float() * target.float()).sum()
    union = (pred.float() + target.float()).clamp(0, 1).sum()
    return (intersection + smooth) / (union + smooth)

# Dice Score
def dice_score(pred, target, smooth=1e-6):
    pred = torch.sigmoid(pred) > 0.5
    target = target > 0.5
    intersection = (pred.float() * target.float()).sum()
    return (2. * intersection + smooth) / (pred.float().sum() + target.float().sum() + smooth)

# Training Loop
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    best_iou = 0.0
    train_loss_history = []
    train_iou_history = []
    train_dice_history = []

    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    for epoch in range(num_epochs):
        epoch_loss = 0
        epoch_iou = 0
        epoch_dice = 0
        correct = 0
        total = 0

        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()
            outputs = model(images)  # No ['out'] for ResUNet++
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_iou += iou_score(outputs, masks).item()
            epoch_dice += dice_score(outputs, masks).item()

            predicted = torch.sigmoid(outputs) > 0.5
            correct += (predicted == masks).sum().item()
            total += masks.numel()

        accuracy = correct / total
        epoch_loss /= len(dataloader)
        epoch_iou /= len(dataloader)
        epoch_dice /= len(dataloader)

        train_loss_history.append(epoch_loss)
        train_iou_history.append(epoch_iou)
        train_dice_history.append(epoch_dice)

        # Save the best model based on IoU score
        if epoch_iou > best_iou:
            best_iou = epoch_iou
            torch.save(model.state_dict(), "best_resunetpp.pth")

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, IoU: {epoch_iou:.4f}, Dice: {epoch_dice:.4f}, Accuracy: {accuracy:.4f}")

    print("Training complete.")

    # Plot the metrics
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.plot(train_loss_history, label="Loss")
    plt.title("Loss over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")

    plt.subplot(1, 3, 2)
    plt.plot(train_iou_history, label="IoU", color="orange")
    plt.title("IoU over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("IoU")

    plt.subplot(1, 3, 3)
    plt.plot(train_dice_history, label="Dice", color="green")
    plt.title("Dice over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Dice")

    plt.tight_layout()
    plt.show()

# Assuming your model, dataloader, criterion, and optimizer are already defined
train_model(model, train_loader, criterion, optimizer, num_epochs=10)


Epoch 1/10, Loss: 0.3654, IoU: 0.0873, Dice: 0.1466, Accuracy: 0.8336
